In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

I'm interested in seeing which metrics are most usefull, and follow the utility score. Most people default to AUC in classification problems, however, here prediction ordering does not matter, so AUC might not be ideal. 

In [ ]:
train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.query('weight>0').reset_index(drop=True)

train.fillna(train.median(),inplace=True)

In [ ]:
features = [c for c in train.columns if 'feature' in c]

In [ ]:
resp_cols = ['resp']

In [ ]:
import tensorflow_addons as tfa
import tensorflow as tf
import tensorflow.keras.backend as K

def create_model(metric):
    
    model = tf.keras.models.Sequential(
    [
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1,activation='sigmoid'),
    ]
    )
    optimizer = 'adam'
    model.compile(optimizer, loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.001),metrics=[metric])
    return model

In [ ]:
def utility_score_bincount(date, weight, resp, action):
    count_i = len(np.unique(date))
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return u

In [ ]:
import random
import os
def set_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
set_all_seeds(42)

In [ ]:
from sklearn.model_selection import GroupKFold
from collections import defaultdict

out = defaultdict(list)
X = train[features].values
y = (train['resp'].values > 0).astype(int)

folds = list(GroupKFold(10).split(X,y,groups=train['date'].values))

for metric in [tf.keras.metrics.AUC(name='auc'),
              tf.keras.metrics.BinaryAccuracy(name='accuracy'),
              tf.keras.metrics.Precision(name='precision'),
              tf.keras.metrics.Recall(name='recall'),
              tf.keras.metrics.PrecisionAtRecall(0.55, name='patr'),
              tf.keras.metrics.RecallAtPrecision(0.55, name='ratp')
              ]:

    for tr,te in folds:
        X_train, X_test = X[tr],X[te]
        y_train, y_test = y[tr],y[te]

        model = create_model(metric)
        hist = model.fit(X_train,y_train,
                         validation_data=(X_test,y_test),
                         epochs=40,
                         batch_size=8192,
                         callbacks=[tf.keras.callbacks.EarlyStopping(f'val_{metric.name}',
                                                                     mode='max',
                                                                     patience=3,
                                                                     restore_best_weights=True)])
        
        y_pred = model.predict(X_test).reshape((-1,))
        out['us'].append(utility_score_bincount(train['date'].values[te], train['weight'].values[te], train['resp'].values[te], np.where(y_pred>0.5,1,0)))
        out[metric.name].append(max(hist.history[f'val_{metric.name}']))


In [ ]:
out['us'] = np.mean(np.asarray(out['us']).reshape((6,len(folds))),axis=0)
for k in out:
    out[k] = np.asarray(out[k])

In [ ]:
import matplotlib.pyplot as plt
dataframe=pd.DataFrame(data=out)
corr = dataframe.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)